# Import Libraries

In [1]:
import sqlite3 
import pandas as pd
from sklearn.tree import DecisionTreeRegressor 
from sklearn.linear_model import LinearRegression 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error 
from math import sqrt

# Read Data from the Database into pandas

In [2]:
# Create your connection.
cnx = sqlite3.connect('database.sqlite')
df_1 = pd.read_sql_query("SELECT * FROM Player_Attributes", cnx)

In [3]:
df_1.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [4]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183978 entries, 0 to 183977
Data columns (total 42 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   183978 non-null  int64  
 1   player_fifa_api_id   183978 non-null  int64  
 2   player_api_id        183978 non-null  int64  
 3   date                 183978 non-null  object 
 4   overall_rating       183142 non-null  float64
 5   potential            183142 non-null  float64
 6   preferred_foot       183142 non-null  object 
 7   attacking_work_rate  180748 non-null  object 
 8   defensive_work_rate  183142 non-null  object 
 9   crossing             183142 non-null  float64
 10  finishing            183142 non-null  float64
 11  heading_accuracy     183142 non-null  float64
 12  short_passing        183142 non-null  float64
 13  volleys              181265 non-null  float64
 14  dribbling            183142 non-null  float64
 15  curve            

In [5]:
df=df_1.dropna()

In [6]:
df=df.drop(['id','player_fifa_api_id','player_api_id','date'], axis=1)

In [7]:
df.shape

(180354, 38)

In [8]:
df.head()

,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,67.0,71.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,67.0,71.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,62.0,66.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,61.0,65.0,right,medium,medium,48.0,43.0,70.0,60.0,43.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,61.0,65.0,right,medium,medium,48.0,43.0,70.0,60.0,43.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [9]:
df['preferred_foot'].value_counts()

right    136247
left      44107
Name: preferred_foot, dtype: int64

In [10]:
df['attacking_work_rate'].value_counts()

medium    125070
high       42823
low         8569
None        3317
norm         317
y             94
stoc          86
le            78
Name: attacking_work_rate, dtype: int64

In [11]:
df=df[(df['attacking_work_rate']!= 'norm') & (df['attacking_work_rate']!= 'y') & (df['attacking_work_rate']!= 'stoc') & (df['attacking_work_rate']!= 'le')]

In [12]:
df['attacking_work_rate'].value_counts()

medium    125070
high       42823
low         8569
None        3317
Name: attacking_work_rate, dtype: int64

In [13]:
df.shape

(179779, 38)

In [14]:
df['defensive_work_rate'].value_counts()

medium    130846
high       27041
low        18432
o           1328
1            421
2            334
3            243
5            231
7            207
0            188
6            179
9            143
4            116
8             70
Name: defensive_work_rate, dtype: int64

In [15]:
df=df[(df['defensive_work_rate']=='medium') | (df['defensive_work_rate']=='high') | (df['defensive_work_rate']=='low')]

In [16]:
df.shape

(176319, 38)

In [17]:
preferred_foot = pd.get_dummies(df['preferred_foot'], drop_first=True)
attacking_work_rate=pd.get_dummies(df['attacking_work_rate'], drop_first=True)
defensive_work_rate=pd.get_dummies(df['defensive_work_rate'], drop_first=True)

In [18]:
df=pd.concat([df, preferred_foot, attacking_work_rate, defensive_work_rate], axis=1)

In [19]:
df.head()

,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,...,gk_handling,gk_kicking,gk_positioning,gk_reflexes,right,high,low,medium,low,medium
0,67.0,71.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,...,11.0,10.0,8.0,8.0,1,0,0,1,0,1
1,67.0,71.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,...,11.0,10.0,8.0,8.0,1,0,0,1,0,1
2,62.0,66.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,...,11.0,10.0,8.0,8.0,1,0,0,1,0,1
3,61.0,65.0,right,medium,medium,48.0,43.0,70.0,60.0,43.0,...,10.0,9.0,7.0,7.0,1,0,0,1,0,1
4,61.0,65.0,right,medium,medium,48.0,43.0,70.0,60.0,43.0,...,10.0,9.0,7.0,7.0,1,0,0,1,0,1


In [20]:
df.drop(['preferred_foot', 'attacking_work_rate', 'defensive_work_rate'], axis=1, inplace=True)

In [21]:
df.head()

,overall_rating,potential,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,...,gk_handling,gk_kicking,gk_positioning,gk_reflexes,right,high,low,medium,low,medium
0,67.0,71.0,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,...,11.0,10.0,8.0,8.0,1,0,0,1,0,1
1,67.0,71.0,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,...,11.0,10.0,8.0,8.0,1,0,0,1,0,1
2,62.0,66.0,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,...,11.0,10.0,8.0,8.0,1,0,0,1,0,1
3,61.0,65.0,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,...,10.0,9.0,7.0,7.0,1,0,0,1,0,1
4,61.0,65.0,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,...,10.0,9.0,7.0,7.0,1,0,0,1,0,1


In [22]:
y=df['overall_rating']
features=df.drop(['overall_rating'], axis=1)

In [23]:
from sklearn import preprocessing

In [24]:
min_max_scaler=preprocessing.MinMaxScaler()

In [25]:
scaled_features=min_max_scaler.fit_transform(features)

In [26]:
from sklearn.decomposition import PCA
pca = PCA(.95)
pca.fit(scaled_features)

PCA(n_components=0.95)

In [27]:
features_1=pca.transform(scaled_features)

In [28]:
features_1.shape

(176319, 17)

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features_1, y, test_size=0.25, random_state=100)

In [30]:
from xgboost import XGBRegressor
XGB=XGBRegressor()

In [31]:
XGB.fit(X_train,y_train)

[23:17:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor()

In [32]:
y_pred_train = XGB.predict(X_train)

In [33]:
from sklearn.metrics import r2_score

In [34]:
score=r2_score(y_train, y_pred_train)
score

0.8705664578945638

In [35]:
y_pred_test = XGB.predict(X_test)
score=r2_score(y_test, y_pred_test)
score

0.8661186919869349

In [36]:
from sklearn.model_selection import GridSearchCV

In [37]:
param_grid={
   
    ' learning_rate':[1,0.5,0.1,0.01,0.001],
    'max_depth': [3,5,10,20],
    'n_estimators':[10,50,100,200]
    
}

In [38]:
grid= GridSearchCV(XGBRegressor(),param_grid, verbose=3)

In [39]:
grid.fit(X_train, y_train)

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[CV]  learning_rate=1, max_depth=3, n_estimators=10 ..................
[23:20:57] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]   learning_rate=1, max_depth=3, n_estimators=10, score=-10.827, total=   2.1s
[CV]  learning_rate=1, max_depth=3, n_estimators=10 ..................
[23:20:59] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.1s remaining:    0.0s


[CV]   learning_rate=1, max_depth=3, n_estimators=10, score=-11.007, total=   2.2s
[CV]  learning_rate=1, max_depth=3, n_estimators=10 ..................
[23:21:01] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


...

[CV]   learning_rate=0.5, max_depth=10, n_estimators=50, score=0.952, total=  32.3s
[CV]  learning_rate=0.5, max_depth=10, n_estimators=50 ...............
[00:51:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[CV]   learning_rate=0.5, max_depth=10, n_estimators=50, score=0.952, total=  32.3s
[CV]  learning_rate=0.5, max_depth=10, n_estimators=50 ...............
[00:52:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[CV]   learning_rate=0.5, max_depth=10, n_estimators=50, score=0.953, total=  31.9s
[CV]  learning_rate=0.5, max_depth=10, n_estimators=50 ...............
[00:52:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[CV]   learning_rate=0.5, max_depth=10, n_estimators=50, score=0.952, total=  32.8s
[CV]  learning_rate=0.5, max_depth=10, n_estimators=100 ..............
[00:53:21] WARNING: src/objectiv

[CV]   learning_rate=0.5, max_depth=20, n_estimators=200, score=0.976, total= 4.5min
[CV]  learning_rate=0.5, max_depth=20, n_estimators=200 ..............
[01:39:23] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[CV]   learning_rate=0.5, max_depth=20, n_estimators=200, score=0.977, total= 4.4min
[CV]  learning_rate=0.5, max_depth=20, n_estimators=200 ..............
[01:43:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[CV]   learning_rate=0.5, max_depth=20, n_estimators=200, score=0.975, total= 4.3min
[CV]  learning_rate=0.1, max_depth=3, n_estimators=10 ................
[01:47:59] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[CV]   learning_rate=0.1, max_depth=3, n_estimators=10, score=-10.827, total=   2.0s
[CV]  learning_rate=0.1, max_depth=3, n_estimators=10 ................
[01:48:01] WARNING: src/obje

[CV]   learning_rate=0.1, max_depth=5, n_estimators=50, score=0.890, total=  33.2s
[CV]  learning_rate=0.1, max_depth=5, n_estimators=50 ................
[02:00:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[CV]   learning_rate=0.1, max_depth=5, n_estimators=50, score=0.884, total=  33.3s
[CV]  learning_rate=0.1, max_depth=5, n_estimators=100 ...............
[02:00:41] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[CV]   learning_rate=0.1, max_depth=5, n_estimators=100, score=0.923, total= 1.1min
[CV]  learning_rate=0.1, max_depth=5, n_estimators=100 ...............
[02:01:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[CV]   learning_rate=0.1, max_depth=5, n_estimators=100, score=0.923, total= 1.1min
[CV]  learning_rate=0.1, max_depth=5, n_estimators=100 ...............
[02:02:54] WARNING: src/objective/

[CV]   learning_rate=0.1, max_depth=10, n_estimators=200, score=0.969, total= 2.1min
[CV]  learning_rate=0.1, max_depth=20, n_estimators=10 ...............
[09:15:49] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[CV]   learning_rate=0.1, max_depth=20, n_estimators=10, score=-10.626, total=   6.1s
[CV]  learning_rate=0.1, max_depth=20, n_estimators=10 ...............
[09:15:55] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[CV]   learning_rate=0.1, max_depth=20, n_estimators=10, score=-10.798, total=   6.0s
[CV]  learning_rate=0.1, max_depth=20, n_estimators=10 ...............
[09:16:01] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[CV]   learning_rate=0.1, max_depth=20, n_estimators=10, score=-10.824, total=   6.2s
[CV]  learning_rate=0.1, max_depth=20, n_estimators=10 ...............
[09:16:07] WARNING: src/o

KeyboardInterrupt: 

In [ ]:
grid.best_params_

In [ ]:
new_model=XGBRegressor(learning_rate= 1, max_depth= 5, n_estimators= 50)
new_model.fit(X_train, y_train)

In [ ]:
y_pred_train = new_model.predict(X_train)
score=r2_score(y_train, y_pred_train)
score

In [ ]:
y_pred_test = new_model.predict(X_test)
score=r2_score(y_test, y_pred_test)
score